In [ ]:
import requests, json
from bs4 import BeautifulSoup

In [ ]:
# scraping the job grouplists
rooturl = "https://www.nsp.cz/odborny-smer"
rootpage = requests.get(rooturl).text
rootsoup = BeautifulSoup(rootpage, 'html.parser')
sel = rootsoup.select(".bricklayer li a")
rootlinks = [a["href"] for a in sel]

In [ ]:
jobdict = {}

In [ ]:
for link in rootlinks:
    if link == "https://www.nsp.cz/odborny-podsmer-rozsireny/ozbrojene-sily-a-bezpecnostni-sbory":
        link = "https://www.nsp.cz/odborny-smer/ozbrojene-sily-a-bezpecnostni-sbory"
    print(link)
    group_json = requests.get(link + "?view=0&_format=json", headers={"x-requested-with": "XMLHttpRequest"}).json()
    for entry in group_json['data']:
        for subentry in entry['items']['s1']:
            if subentry['children']:
                for child in subentry['children']:
                    jobdict[child['title']] = {'url': child['urlSlug']}
            else:
                jobdict[subentry['title']] = {'url': subentry['urlSlug']}

In [ ]:
# scraping the individual pages
print(len(jobdict))
i = 0
for job in jobdict:
    i += 1
    print(i, end=" ")
    subsrc = requests.get("https://www.nsp.cz/jednotka-prace/" + jobdict[job]['url']).text
    subsrcsoup = BeautifulSoup(subsrc, 'html.parser')
    jobsrc = subsrcsoup.select(".search-results__results > li")
    jobdict[job]["skills"] = {}
    skills = {"odb_dov": "#TYPE_TECHNICAL_HARD_SKILL", "odb_zna": "#TYPE_SPECIFIC_HARD_SKILL", "obecne": "#TYPE_GENERIC_HARD_SKILL", "mekke": "#TYPE_SOFT_SKILL"}
    for skill in skills:
        subskills = {}
        skillsoup = subsrcsoup.select(skills[skill]+" tbody tr")

        for subskill in skillsoup:
            subskills[subskill.get_text().strip()] = len(subskill.select(".circle--green"))
        jobdict[job]["skills"][skill] = subskills

In [ ]:
for job in jobdict:
    if len(jobdict[job]["skills"]) == 0:
        print(jobdict[job])

In [25]:
with open("data/jobs.json","w",encoding="utf-8") as file:
    file.write(json.dumps(jobdict, ensure_ascii=False))